# SingGlow Experiment

In [1]:
%conda activate SinGlow


Note: you may need to restart the kernel to use updated packages.



CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
If using 'conda activate' from a batch script, change your
invocation to 'CALL conda.bat activate'.

To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - cmd.exe
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [2]:
from common_definitions import *
from pipeline import *
import tensorflow as tf
import matplotlib.pyplot as plt
import librosa
import librosa.display
import soundfile as sf
import IPython.display as ipd
from tqdm.notebook import tqdm
import pickle
from data_loarder import *

os.chdir(r'./runs')

In [3]:
tfrecord_dir = "./runs"

data_loader = SongDataLoader('real.tfrecords',tfrecord_dir=tfrecord_dir)
data_loader.make("../SongDatabase/RealSinger")
# real_dataset = data_loader.load()
real_dataset = data_loader.load(sampling_num=200)
# del data_loader

data_loader = SongDataLoader('virtual.tfrecords',tfrecord_dir=tfrecord_dir)
data_loader.make("../SongDatabase/VirtualSinger")
virtual_dataset = data_loader.load(sampling_num=200)
# del data_loader

In [4]:
# Step ?. the brain
brain = Brain(SQUEEZE_FACTOR, K_GLOW, L_GLOW, WINDOW_LENGTH, CHANNEL_SIZE, LEARNING_RATE)

# load weight if available
brain.model(tf.random.uniform((2, WINDOW_LENGTH, 1, CHANNEL_SIZE), 0.05, 1), training=True)
# CHECKPOINT_PATH = r'D:\PlayGround\research\SinGlow\checkpoints\weights'
# print(brain.load_weights(CHECKPOINT_PATH))

# Canceled future for execute_request message before replies were done
# The Kernel crashed while executing code in the the current cell or a previous cell. Please review the code in the cell(s) to identify a possible cause of the failure. Click here for more info. View Jupyter log for further details.

: 

: 

In [5]:
import pickle

# real z

real_z_path = "./real_z.pickle"
if os.path.exists(real_z_path):
    with open(real_z_path,mode='rb') as f:
        real_z = pickle.load(f)
else:
    real_z_results = []
    for i in tqdm(real_dataset):
        real_z_results.append(brain.forward(i).numpy()[0])
    real_z = np.apply_along_axis(np.mean,0,np.array(real_z_results))
    with open(real_z_path,mode='wb') as f:
        pickle.dump(real_z, f)

# # virtual z
# if os.path.exists('virtual_z.pickle'):
#     with open('virtual_z.pickle',mode='rb') as f:
#         virtual_z = pickle.load(f)
# else:
#     virtual_z_results = []
#     for i in tqdm(virtual_dataset):
#         virtual_z_results.append(brain.forward(i).numpy()[0])

#     virtual_z = np.apply_along_axis(np.mean,0,np.array(virtual_z_results))
#     with open('virtual_z.pickle',mode='wb') as f:
#         pickle.dump(virtual_z, f)

  0%|          | 0/7 [00:00<?, ?it/s]


NameError: name 'brain' is not defined

In [6]:
# # delta z
# delta_z = real_z-virtual_z

In [7]:
# figure, ax = plt.subplots(3)
# figure.set_size_inches(12,9)
# plt.subplots_adjust(hspace=1)

# ax[0] = plt.plot(np.array(real_z))
# # ax[1] = plt.plot(np.array(delta_z))
# # ax[2] = plt.plot(np.array(virtual_z))

# # librosa.display.waveplot(np.array(real_z), sr=SAMPLING_RATE, ax=ax[0])
# # librosa.display.waveplot(np.array(delta_z), sr=SAMPLING_RATE, ax=ax[1])
# # librosa.display.waveplot(np.array(virtual_z), sr=SAMPLING_RATE, ax=ax[2])

# ax[0].set_title("real_z")
# # ax[1].set_title("delta_z")
# # ax[2].set_title("virtual_z")
# ax[0].set_ylim([-1,1])
# # ax[1].set_ylim([-1,1])
# # ax[2].set_ylim([-1,1])
# plt.show()

In [8]:
os.chdir(r'D:\PlayGround\research\SinGlow\runs')

virtual_file_dir = r'D:\PlayGround\research\SongDatabase\TestSongs'
name = 'virtual_align_short'
pickle_file  = f'result_{name}.pickle'
if os.path.exists(pickle_file):
    with open(pickle_file, mode='rb') as f:
        y, sr = pickle.load(f)
else:
    y, sr = librosa.load(os.path.join(virtual_file_dir, name + '.mp3'))
    with open(pickle_file, mode='wb') as f:
        pickle.dump((y, sr), f)

C:\Users\hobar\anaconda3\envs\DeepLearningTF2\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [9]:
ys = np.array([y[i*sr*WINDOW_SIZE:(i+1)*sr*WINDOW_SIZE] for i in range(len(y)//(sr*WINDOW_SIZE))] + [y[-sr*WINDOW_SIZE:]]).reshape((-1,sr*WINDOW_SIZE,1,1))
ys = tf.image.resize(ys,[WINDOW_LENGTH,1]).numpy().reshape((-1,1,WINDOW_LENGTH,1,1))
ys_dataset = tf.data.Dataset.from_tensor_slices(ys)

In [10]:
result_ys = []
for i in tqdm(ys_dataset):
    result_z = (brain.forward(i)+real_z)/2 # 向量加法中点
    result_ys+=list(tf.squeeze(brain.backward(result_z).numpy()))
sf.write(f'result_{name}.wav', np.array(result_ys), SAMPLING_RATE, subtype='PCM_24')

  0%|          | 0/47 [00:00<?, ?it/s]C:\Users\hobar\anaconda3\envs\DeepLearningTF2\lib\site-packages\keras\legacy_tf_layers\core.py:513: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
C:\Users\hobar\anaconda3\envs\DeepLearningTF2\lib\site-packages\keras\engine\base_layer.py:2215: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
100%|██████████| 47/47 [02:11<00:00,  2.81s/it]


## Test song merge

In [ ]:
# test_file_dir = r'D:\PlayGround\research\SongDatabase\TestSongs'
# name='virtual_align_short'
# virtual_file_path = os.path.join(test_file_dir,name+'.pickle')
# if os.path.exists(virtual_file_path):
#     with open(virtual_file_path,mode='rb') as f:
#         y = pickle.load(f)
# else:
#     y, sr = librosa.load(os.path.join(test_file_dir,name+'.mp3'))
#     with open(virtual_file_path,mode='wb') as f:
#         pickle.dump(y, f)
# virtual_data = y

In [ ]:
# name='real_short_pure_reference'
# real_file_path = os.path.join(test_file_dir,name+'.pickle')
# if os.path.exists(real_file_path):
#     with open(real_file_path,mode='rb') as f:
#         y = pickle.load(f)
# else:
#     y, sr = librosa.load(os.path.join(test_file_dir,name+'.mp3'))
#     with open(real_file_path,mode='wb') as f:
#         pickle.dump(y, f)
# real_data = y

In [ ]:
# real = np.array([real_data[i*22050*WINDOW_SIZE:(i+1)*22050*WINDOW_SIZE] for i in range(len(real_data)//(22050*WINDOW_SIZE))] + [real_data[-22050*WINDOW_SIZE:]]).reshape((-1,22050*WINDOW_SIZE,1,1))
# real = tf.image.resize(real,[WINDOW_LENGTH,1]).numpy().reshape((-1,1,WINDOW_LENGTH,1,1))

# virtual = np.array([virtual_data[i*22050*WINDOW_SIZE:(i+1)*22050*WINDOW_SIZE] for i in range(len(virtual_data)//(22050*WINDOW_SIZE))] + [real_data[-22050*WINDOW_SIZE:]]).reshape((-1,22050*WINDOW_SIZE,1,1))
# virtual = tf.image.resize(virtual,[WINDOW_LENGTH,1]).numpy().reshape((-1,1,WINDOW_LENGTH,1,1))

# ys_dataset = tf.data.Dataset.from_tensor_slices((virtual,real[:virtual.shape[0]]))

In [ ]:
# result_ys = []
# for virtual,real in tqdm(ys_dataset):
#     virtual_forward = brain.forward(virtual)
#     real_forward = brain.forward(real)
#     result_z = virtual_forward/2 + real_forward/2
#     result_ys+=list(tf.clip_by_value(tf.squeeze(brain.backward(result_z)),-1,1).numpy())
# virtual_file_path = os.path.join(test_file_dir,'result.wav')
# sf.write(virtual_file_path, np.array(result_ys), SAMPLING_RATE, subtype='PCM_24')